**This Notebook provides an ensemble model that combines the inference output from EffecientNet model and 2.5D s-stage model.**

## Load Package

In [222]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
from time import time
from datetime import datetime
from path import Path
from sklearn.model_selection import train_test_split
import random
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    fbeta_score,
    recall_score,
    precision_score,
)


import warnings
warnings.filterwarnings('ignore')

## Import inference output

#### 2-stage model output

In [7]:
two_stage = pd.read_csv('valid_folds_all.csv')
two_stage['prediction_type'] = 'C'+two_stage['cid'].astype(str)
two_stage

,study_cid,StudyInstanceUID,cid,slice_num_list,before_image_size,x0,x1,y0,y1,z0,z1,label,Preds,Trues,prediction_type
0,1.2.826.0.1.3680043.10001_1,1.2.826.0.1.3680043.10001,1,"[50, 51, 52, 53, 55, 56, 57, 59, 60, 61, 63, 6...",320,0,267,2,273,48,277,0,0.113253,0,C1
1,1.2.826.0.1.3680043.10001_2,1.2.826.0.1.3680043.10001,2,"[1, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92...",320,0,267,2,273,48,277,0,0.192726,0,C2
2,1.2.826.0.1.3680043.10001_3,1.2.826.0.1.3680043.10001,3,"[108, 109, 110, 111, 112, 113, 114, 115, 116, ...",320,0,267,2,273,48,277,0,0.092990,0,C3
3,1.2.826.0.1.3680043.10001_4,1.2.826.0.1.3680043.10001,4,"[129, 130, 131, 132, 133, 134, 135, 136, 137, ...",320,0,267,2,273,48,277,0,0.079985,0,C4
4,1.2.826.0.1.3680043.10001_5,1.2.826.0.1.3680043.10001,5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",320,0,267,2,273,48,277,0,0.123712,0,C5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13555,1.2.826.0.1.3680043.9997_3,1.2.826.0.1.3680043.9997,3,"[93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 10...",320,0,254,2,262,3,252,0,0.166747,0,C3
13556,1.2.826.0.1.3680043.9997_4,1.2.826.0.1.3680043.9997,4,"[114, 115, 116, 117, 118, 119, 120, 121, 122, ...",320,0,254,2,262,3,252,0,0.150317,0,C4
13557,1.2.826.0.1.3680043.9997_5,1.2.826.0.1.3680043.9997,5,"[136, 137, 138, 139, 140, 141, 142, 144, 145, ...",320,0,254,2,262,3,252,0,0.377570,0,C5
13558,1.2.826.0.1.3680043.9997_6,1.2.826.0.1.3680043.9997,6,"[158, 159, 160, 161, 162, 163, 164, 165, 166, ...",320,0,254,2,262,3,252,0,0.838126,0,C6


#### EfficientNet Output

In [3]:
effnet = pd.read_csv('effent_inference_500_patients.csv')
effnet

,actual,row_id,StudyInstanceUID,prediction_type,fractured,inference_time,pred_flag,Accuracy
0,1,1.2.826.0.1.3680043.6200_patient_overall,1.2.826.0.1.3680043.6200,patient_overall,0.132991,12.565742,0,0
1,1,1.2.826.0.1.3680043.6200_C1,1.2.826.0.1.3680043.6200,C1,0.315373,12.764569,0,0
2,1,1.2.826.0.1.3680043.6200_C2,1.2.826.0.1.3680043.6200,C2,0.423518,10.976760,1,1
3,0,1.2.826.0.1.3680043.6200_C3,1.2.826.0.1.3680043.6200,C3,0.095354,14.120935,0,1
4,0,1.2.826.0.1.3680043.6200_C4,1.2.826.0.1.3680043.6200,C4,0.333067,10.120789,0,1
...,...,...,...,...,...,...,...,...
3995,0,1.2.826.0.1.3680043.14654_C3,1.2.826.0.1.3680043.14654,C3,0.348316,42.737215,1,0
3996,0,1.2.826.0.1.3680043.14654_C4,1.2.826.0.1.3680043.14654,C4,0.296459,42.737215,0,1
3997,0,1.2.826.0.1.3680043.14654_C5,1.2.826.0.1.3680043.14654,C5,0.786132,42.737215,1,0
3998,0,1.2.826.0.1.3680043.14654_C6,1.2.826.0.1.3680043.14654,C6,0.186581,42.737215,0,1


### Combines output of 2 models

In [135]:
overall_infer = two_stage.merge(effnet, how='inner',
                                left_on = ['StudyInstanceUID','prediction_type'], 
                                right_on = ['StudyInstanceUID','prediction_type'])
overall_infer['two_stage_pred'] = overall_infer['Preds']
overall_infer['effnet_pred'] = overall_infer['fractured']
overall_infer = overall_infer[['StudyInstanceUID','prediction_type','cid','x0','x1',
                               'y0','y1','z0','z1','label','two_stage_pred','effnet_pred']]

overall_infer

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892
1,1.2.826.0.1.3680043.10032,C2,2,0,320,1,314,72,243,0,0.192151,0.412402
2,1.2.826.0.1.3680043.10032,C3,3,0,320,1,314,72,243,0,0.083235,0.345822
3,1.2.826.0.1.3680043.10032,C4,4,0,320,1,314,72,243,0,0.153492,0.384723
4,1.2.826.0.1.3680043.10032,C5,5,0,320,1,314,72,243,0,0.175922,0.816983
...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9996,C3,3,0,295,3,320,72,254,0,0.083757,0.114156
3358,1.2.826.0.1.3680043.9996,C4,4,0,295,3,320,72,254,0,0.129619,0.344138
3359,1.2.826.0.1.3680043.9996,C5,5,0,295,3,320,72,254,0,0.117988,0.473295
3360,1.2.826.0.1.3680043.9996,C6,6,0,295,3,320,72,254,0,0.319698,0.275896


#### Add the previous calculated classification threshold

In [136]:
threshold = {'prediction_type': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
             'two_stage_threshold': [0.23, 0.34, 0.23, 0.15, 0.19, 0.23, 0.42],
             'effnet_threshold': [0.67, 0.42, 0.31, 0.34, 0.73, 0.21, 0.81],
            }
  

two_stage_threshold = pd.DataFrame(threshold)
overall_infer = overall_infer.merge(two_stage_threshold, how='inner',
                                left_on = 'prediction_type', 
                                right_on = 'prediction_type')
overall_infer

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred,two_stage_threshold,effnet_threshold
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892,0.23,0.67
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,0.091811,0.332340,0.23,0.67
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,0.124926,0.695181,0.23,0.67
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,0.120831,0.685851,0.23,0.67
4,1.2.826.0.1.3680043.10443,C1,1,0,601,2,247,84,248,0,0.218730,0.683365,0.23,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,0.388187,0.791127,0.42,0.81
3358,1.2.826.0.1.3680043.9921,C7,7,0,261,0,317,2,314,1,0.536955,0.847256,0.42,0.81
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,0.437208,0.195100,0.42,0.81
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,0.613176,0.811858,0.42,0.81


#### Add patient overall rows

In [132]:
patient_id = overall_infer['StudyInstanceUID'].drop_duplicates()
train_id = np.random.choice(patient_id,400, replace=False)

In [133]:
train_label = {'StudyInstanceUID': train_id,
             'train': [1]*len(train_id)
            }
train = pd.DataFrame(train_label)

,StudyInstanceUID,train
0,1.2.826.0.1.3680043.1381,1
1,1.2.826.0.1.3680043.4270,1
2,1.2.826.0.1.3680043.12666,1
3,1.2.826.0.1.3680043.29351,1
4,1.2.826.0.1.3680043.20038,1
...,...,...
395,1.2.826.0.1.3680043.21350,1
396,1.2.826.0.1.3680043.28968,1
397,1.2.826.0.1.3680043.3130,1
398,1.2.826.0.1.3680043.9548,1


In [137]:
overall_infer = overall_infer.merge(train, how='left',
                                left_on = 'StudyInstanceUID', 
                                right_on = 'StudyInstanceUID').drop_duplicates()
overall_infer.loc[overall_infer["train"] != 1, "train"] = 0
overall_infer

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred,two_stage_threshold,effnet_threshold,train
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892,0.23,0.67,1.0
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,0.091811,0.332340,0.23,0.67,1.0
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,0.124926,0.695181,0.23,0.67,1.0
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,0.120831,0.685851,0.23,0.67,1.0
4,1.2.826.0.1.3680043.10443,C1,1,0,601,2,247,84,248,0,0.218730,0.683365,0.23,0.67,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,0.388187,0.791127,0.42,0.81,1.0
3358,1.2.826.0.1.3680043.9921,C7,7,0,261,0,317,2,314,1,0.536955,0.847256,0.42,0.81,0.0
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,0.437208,0.195100,0.42,0.81,1.0
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,0.613176,0.811858,0.42,0.81,1.0


#### Save combined inference

In [138]:
overall_infer.to_csv('overall_infer.csv', index=False)

## Train Test Split

In [125]:
overall_infer = pd.read_csv('overall_infer.csv')

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,two_stage_pred,effnet_pred,two_stage_threshold,effnet_threshold,train
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,0.114832,0.691892,0.23,0.67,0.0
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,0.091811,0.332340,0.23,0.67,1.0
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,0.124926,0.695181,0.23,0.67,0.0
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,0.120831,0.685851,0.23,0.67,1.0
4,1.2.826.0.1.3680043.10443,C1,1,0,601,2,247,84,248,0,0.218730,0.683365,0.23,0.67,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,0.388187,0.791127,0.42,0.81,1.0
3358,1.2.826.0.1.3680043.9921,C7,7,0,261,0,317,2,314,1,0.536955,0.847256,0.42,0.81,1.0
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,0.437208,0.195100,0.42,0.81,1.0
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,0.613176,0.811858,0.42,0.81,0.0


In [279]:
overall_infer_train = overall_infer[overall_infer['train'] == 1]
overall_infer_test = overall_infer[overall_infer['train'] == 0]

## Ensemble wtih various weights

#### Add model weight

In [280]:
two_stage_weight = [0.7,0.75,0.8,0.85,0.9,0.95]
for i in two_stage_weight:
    overall_infer_train[f'two_stage_{int(100*i)}'] = overall_infer_train['two_stage_pred']*i + overall_infer_train['effnet_pred']*(1-i)

overall_infer_train['two_stage_pred_label'] = np.where(overall_infer_train['two_stage_pred']>=overall_infer_train['two_stage_threshold'],1,0)
overall_infer_train['effnet_pred_label'] = np.where(overall_infer_train['effnet_pred']>=overall_infer_train['effnet_threshold'],1,0)
   
overall_infer_train

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,...,effnet_threshold,train,two_stage_pred_label,effnet_pred_label,two_stage_70,two_stage_75,two_stage_80,two_stage_85,two_stage_90,two_stage_95
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,...,0.67,1.0,0,1,0.287950,0.259097,0.230244,0.201391,0.172538,0.143685
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,...,0.67,1.0,0,0,0.163969,0.151943,0.139917,0.127890,0.115864,0.103837
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,...,0.67,1.0,0,1,0.296002,0.267489,0.238977,0.210464,0.181951,0.153438
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,...,0.67,1.0,0,1,0.290337,0.262086,0.233835,0.205584,0.177333,0.149082
5,1.2.826.0.1.3680043.10697,C1,1,0,179,58,309,6,256,0,...,0.67,1.0,0,1,0.379362,0.353331,0.327300,0.301269,0.275238,0.249207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,1.2.826.0.1.3680043.9774,C7,7,38,260,91,268,98,224,0,...,0.81,1.0,1,1,0.620081,0.605647,0.591214,0.576780,0.562347,0.547913
3357,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,...,0.81,1.0,0,0,0.509069,0.488922,0.468775,0.448628,0.428481,0.408334
3359,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,...,0.81,1.0,1,0,0.364575,0.376681,0.388786,0.400892,0.412997,0.425102
3360,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,...,0.81,1.0,1,1,0.672780,0.662846,0.652912,0.642978,0.633044,0.623110


## Evaluation

In [281]:
def define_eval_metrics(df, a, p):
    recall = np.round(recall_score(y_true=df[a], y_pred=df[p]) * 100, 2)
    precision = np.round(
        precision_score(y_true=df[a], y_pred=df[p]) * 100, 2
    )

    tn, fp, fn, tp = confusion_matrix(y_true=df[a], y_pred=df[p]).ravel()

    fpr = np.round(fp / (fp + tn) * 100, 2)
    fnr = np.round(fn / (fn + tp) * 100, 2)

    accuracy = np.round(
        accuracy_score(
            y_true=df[a],
            y_pred=df[p],
        )
        * 100
    )

    f1 = np.round(fbeta_score(y_true=df[a], y_pred=df[p], beta=1) * 100, 2)

    f05 = np.round(
        fbeta_score(y_true=df[a], y_pred=df[p], beta=0.5) * 100, 2
    )

    f2 = np.round(fbeta_score(y_true=df[a], y_pred=df[p], beta=2) * 100, 2)

    return [recall, precision, tn, fp, fn, tp, fpr, fnr, accuracy, f1, f05, f2]

In [285]:
def find_best_f1(df):

    two_stage_weight = [0.7,0.75,0.8,0.85,0.9,0.95]

    label = ['two_stage_70_label','two_stage_75_label',
         'two_stage_80_label','two_stage_85_label','two_stage_90_label','two_stage_95_label']
    
    for j in label:
        best_f2 = [0,0,0,0,0,0,0]
        best_threshold = [0.1,0.1,0.1,0.1,0.1,0.1,0.1]
        
        for curr_threshold in np.linspace(0.1, 0.51, 16):
            for i in two_stage_weight:
                df[f'two_stage_{int(100*i)}_label'] = np.where(df[f'two_stage_{int(100*i)}']>=curr_threshold,1,0)    
            
            metric_value_c1 = define_eval_metrics(df[df['prediction_type']=='C1'],'label', j)
            metric_value_c2 = define_eval_metrics(df[df['prediction_type']=='C2'],'label', j)
            metric_value_c3 = define_eval_metrics(df[df['prediction_type']=='C3'],'label', j)
            metric_value_c4 = define_eval_metrics(df[df['prediction_type']=='C4'],'label', j)
            metric_value_c5 = define_eval_metrics(df[df['prediction_type']=='C5'],'label', j)
            metric_value_c6 = define_eval_metrics(df[df['prediction_type']=='C6'],'label', j)
            metric_value_c7 = define_eval_metrics(df[df['prediction_type']=='C7'],'label', j)

            if metric_value_c1[11]> best_f2[0]:
                best_f2[0] = metric_value_c1[9]
                best_threshold[0] = curr_threshold
            if metric_value_c2[11]> best_f2[1]:
                best_f2[1] = metric_value_c2[9]
                best_threshold[2] = curr_threshold
            if metric_value_c3[11]> best_f2[2]:
                best_f2[2] = metric_value_c3[9]
                best_threshold[3] = curr_threshold
            if metric_value_c4[11]> best_f2[3]:
                best_f2[3] = metric_value_c4[9]
                best_threshold[4] = curr_threshold
            if metric_value_c5[11]> best_f2[4]:
                best_f2[4] = metric_value_c5[9]
                best_threshold[5] = curr_threshold
            if metric_value_c6[11]> best_f2[5]:
                best_f2[5] = metric_value_c6[9]
                best_threshold[6] = curr_threshold
            if metric_value_c7[11]> best_f2[6]:
                best_f2[6] = metric_value_c7[9]
                best_threshold[6] = curr_threshold

        print(j+'/n')
        print(pd.DataFrame(dict(vertebrae=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],best_f2=best_f2,thresdhold=best_threshold)))

#### Get best thredhold for each ensemble model yields best F2

In [286]:
find_best_f1(overall_infer_train)

two_stage_70_label/n
  vertebrae  best_f2  thresdhold
0        C1    40.00    0.455333
1        C2    43.20    0.100000
2        C3    10.81    0.482667
3        C4    11.11    0.373333
4        C5    13.11    0.346000
5        C6    25.20    0.510000
6        C7    27.36    0.510000
two_stage_75_label/n
  vertebrae  best_f2  thresdhold
0        C1    41.94    0.428000
1        C2    44.09    0.100000
2        C3    12.12    0.482667
3        C4    11.11    0.400667
4        C5    14.04    0.346000
5        C6    25.64    0.510000
6        C7    29.58    0.510000
two_stage_80_label/n
  vertebrae  best_f2  thresdhold
0        C1    42.62    0.428000
1        C2    44.96    0.100000
2        C3    10.53    0.482667
3        C4    11.54    0.373333
4        C5    14.81    0.346000
5        C6    25.20    0.510000
6        C7    30.61    0.510000
two_stage_85_label/n
  vertebrae  best_f2  thresdhold
0        C1    42.62    0.428000
1        C2    45.80    0.100000
2        C3    10.26    0

#### Get the predicted label of each ensemble model based on best classification threshold

In [288]:
two_stage_70_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_70_thresdhold= [0.46, 0.1, 0.48,0.37,0.35,0.51,0.51]))
two_stage_75_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_75_thresdhold= [0.43, 0.1, 0.48,0.4,0.35,0.51,0.51]))
two_stage_80_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_80_thresdhold= [0.43, 0.1, 0.48,0.37,0.35,0.51,0.51]))
two_stage_85_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_85_thresdhold= [0.43, 0.1, 0.48,0.37,0.35,0.51,0.51]))
two_stage_90_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_90_thresdhold= [0.46, 0.1, 0.48,0.37,0.35,0.48,0.51]))
two_stage_95_thresdhold = pd.DataFrame(dict(prediction_type=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 
                                            two_stage_95_thresdhold= [0.48, 0.1, 0.51,0.37,0.34,0.40,0.51]))

overall_infer_train = overall_infer_train.merge(two_stage_70_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_75_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_80_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_85_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_90_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train = overall_infer_train.merge(two_stage_95_thresdhold, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')
overall_infer_train

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,...,two_stage_80_label,two_stage_85_label,two_stage_90_label,two_stage_95_label,two_stage_70_thresdhold,two_stage_75_thresdhold,two_stage_80_thresdhold,two_stage_85_thresdhold,two_stage_90_thresdhold,two_stage_95_thresdhold
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
4,1.2.826.0.1.3680043.10697,C1,1,0,179,58,309,6,256,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1.2.826.0.1.3680043.9774,C7,7,38,260,91,268,98,224,0,...,1,1,1,1,0.51,0.51,0.51,0.51,0.51,0.51
2681,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,...,0,0,0,0,0.51,0.51,0.51,0.51,0.51,0.51
2682,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,...,0,0,0,0,0.51,0.51,0.51,0.51,0.51,0.51
2683,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,...,1,1,1,1,0.51,0.51,0.51,0.51,0.51,0.51


In [289]:
two_stage_weight = [0.7,0.75,0.8,0.85,0.9,0.95]
for i in two_stage_weight:
    overall_infer_train[f'two_stage_{int(100*i)}_label'] = np.where(overall_infer_train[f'two_stage_{int(100*i)}']>=overall_infer_train[f'two_stage_{int(100*i)}_thresdhold'],1,0) 

overall_infer_train

,StudyInstanceUID,prediction_type,cid,x0,x1,y0,y1,z0,z1,label,...,two_stage_80_label,two_stage_85_label,two_stage_90_label,two_stage_95_label,two_stage_70_thresdhold,two_stage_75_thresdhold,two_stage_80_thresdhold,two_stage_85_thresdhold,two_stage_90_thresdhold,two_stage_95_thresdhold
0,1.2.826.0.1.3680043.10032,C1,1,0,320,1,314,72,243,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
1,1.2.826.0.1.3680043.10062,C1,1,0,349,1,316,2,262,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
2,1.2.826.0.1.3680043.10136,C1,1,0,357,2,317,76,250,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
3,1.2.826.0.1.3680043.10412,C1,1,0,684,2,216,69,230,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
4,1.2.826.0.1.3680043.10697,C1,1,0,179,58,309,6,256,0,...,0,0,0,0,0.46,0.43,0.43,0.43,0.46,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1.2.826.0.1.3680043.9774,C7,7,38,260,91,268,98,224,0,...,1,1,1,1,0.51,0.51,0.51,0.51,0.51,0.51
2681,1.2.826.0.1.3680043.9809,C7,7,0,568,3,224,82,237,0,...,0,0,0,0,0.51,0.51,0.51,0.51,0.51,0.51
2682,1.2.826.0.1.3680043.9940,C7,7,41,257,40,315,67,281,0,...,0,0,0,0,0.51,0.51,0.51,0.51,0.51,0.51
2683,1.2.826.0.1.3680043.9994,C7,7,0,264,5,320,89,238,0,...,1,1,1,1,0.51,0.51,0.51,0.51,0.51,0.51


#### Get patient over label

In [290]:
overall_infer_train_overall = overall_infer_train.groupby('StudyInstanceUID')[['label','two_stage_pred_label','effnet_pred_label','two_stage_70_label',
                                                       'two_stage_75_label','two_stage_80_label','two_stage_85_label',
                                                       'two_stage_90_label','two_stage_95_label']].sum().reset_index()
overall_infer_train_overall['label'] =  np.where(overall_infer_train_overall['label']>0,1,0)
overall_infer_train_overall['two_stage_pred_label'] =  np.where(overall_infer_train_overall['two_stage_pred_label']>0,1,0)
overall_infer_train_overall['effnet_pred_label'] =  np.where(overall_infer_train_overall['effnet_pred_label']>0,1,0)
overall_infer_train_overall['two_stage_70_label'] =  np.where(overall_infer_train_overall['two_stage_70_label']>0,1,0)
overall_infer_train_overall['two_stage_75_label'] =  np.where(overall_infer_train_overall['two_stage_75_label']>0,1,0)
overall_infer_train_overall['two_stage_80_label'] =  np.where(overall_infer_train_overall['two_stage_80_label']>0,1,0)
overall_infer_train_overall['two_stage_85_label'] =  np.where(overall_infer_train_overall['two_stage_85_label']>0,1,0)
overall_infer_train_overall['two_stage_90_label'] =  np.where(overall_infer_train_overall['two_stage_90_label']>0,1,0)
overall_infer_train_overall['two_stage_95_label'] =  np.where(overall_infer_train_overall['two_stage_95_label']>0,1,0)
overall_infer_train_overall['prediction_type'] = 'patient_overall'
overall_infer_train_overall

,StudyInstanceUID,label,two_stage_pred_label,effnet_pred_label,two_stage_70_label,two_stage_75_label,two_stage_80_label,two_stage_85_label,two_stage_90_label,two_stage_95_label,prediction_type
0,1.2.826.0.1.3680043.10032,0,1,1,1,1,1,1,1,1,patient_overall
1,1.2.826.0.1.3680043.10062,0,1,1,1,1,1,1,1,1,patient_overall
2,1.2.826.0.1.3680043.10136,0,1,1,1,1,1,1,1,1,patient_overall
3,1.2.826.0.1.3680043.10412,0,1,1,1,1,1,1,1,1,patient_overall
4,1.2.826.0.1.3680043.10697,1,1,1,1,1,1,1,1,1,patient_overall
...,...,...,...,...,...,...,...,...,...,...,...
395,1.2.826.0.1.3680043.9774,1,1,1,1,1,1,1,1,1,patient_overall
396,1.2.826.0.1.3680043.9809,0,1,1,1,1,1,1,1,1,patient_overall
397,1.2.826.0.1.3680043.9940,1,1,1,1,1,1,1,1,1,patient_overall
398,1.2.826.0.1.3680043.9994,0,1,1,1,1,1,1,1,1,patient_overall


In [291]:
overall_infer_train_full = pd.concat([overall_infer_train_overall, 
                                      overall_infer_train[['StudyInstanceUID', 'label','two_stage_pred_label','effnet_pred_label','two_stage_70_label',
                                                       'two_stage_75_label','two_stage_80_label','two_stage_85_label',
                                                       'two_stage_90_label','two_stage_95_label','prediction_type']]])
overall_infer_train_full

,StudyInstanceUID,label,two_stage_pred_label,effnet_pred_label,two_stage_70_label,two_stage_75_label,two_stage_80_label,two_stage_85_label,two_stage_90_label,two_stage_95_label,prediction_type
0,1.2.826.0.1.3680043.10032,0,1,1,1,1,1,1,1,1,patient_overall
1,1.2.826.0.1.3680043.10062,0,1,1,1,1,1,1,1,1,patient_overall
2,1.2.826.0.1.3680043.10136,0,1,1,1,1,1,1,1,1,patient_overall
3,1.2.826.0.1.3680043.10412,0,1,1,1,1,1,1,1,1,patient_overall
4,1.2.826.0.1.3680043.10697,1,1,1,1,1,1,1,1,1,patient_overall
...,...,...,...,...,...,...,...,...,...,...,...
2680,1.2.826.0.1.3680043.9774,0,1,1,1,1,1,1,1,1,C7
2681,1.2.826.0.1.3680043.9809,0,0,0,0,0,0,0,0,0,C7
2682,1.2.826.0.1.3680043.9940,0,1,0,0,0,0,0,0,0,C7
2683,1.2.826.0.1.3680043.9994,0,1,1,1,1,1,1,1,1,C7


In [292]:
two_stage_metrix = define_eval_metrics(overall_infer_train, 'label', 'two_stage_pred_label')

#### Evaluation of best weight

In [293]:
model_type = []
metric = []
patient_overall = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []

vertebrae = ['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
label = ['two_stage_pred_label','effnet_pred_label','two_stage_70_label','two_stage_75_label',
         'two_stage_80_label','two_stage_85_label','two_stage_90_label','two_stage_95_label']
metric_name = ['recall', 'precision', 'tn', 'fp', 'fn', 'tp', 'fpr', 'fnr', 'accuracy', 'f1', 'f05', 'f2']
    
for j in label:
        
    metric_value_overall = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='patient_overall'],'label', j)
    metric_value_c1 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C1'],'label', j)
    metric_value_c2 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C2'],'label', j)
    metric_value_c3 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C3'],'label', j)
    metric_value_c4 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C4'],'label', j)
    metric_value_c5 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C5'],'label', j)
    metric_value_c6 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C6'],'label', j)
    metric_value_c7 = define_eval_metrics(overall_infer_train_fill[overall_infer_train_fill['prediction_type']=='C7'],'label', j)
       
    for m in range(12):
        model_type.append(j)
        metric.append(metric_name[m])
        patient_overall.append(metric_value_overall[m])
        c1.append(metric_value_c1[m])
        c2.append(metric_value_c2[m])
        c3.append(metric_value_c3[m])
        c4.append(metric_value_c4[m])
        c5.append(metric_value_c5[m])
        c6.append(metric_value_c6[m])
        c7.append(metric_value_c7[m])


df_metric = pd.DataFrame(dict(model_type=model_type,metric=metric,patient_overall=patient_overall,
                             c1=c1,c2=c2,c3=c3,c4=c4,c5=c5,c6=c6,c7=c7))
df_metric

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,96.67,75.86,86.79,35.71,84.21,67.86,92.00,90.79
1,two_stage_pred_label,precision,45.91,20.00,23.23,7.04,9.41,8.56,15.03,22.85
2,two_stage_pred_label,tn,15.00,270.00,192.00,317.00,218.00,137.00,60.00,66.00
3,two_stage_pred_label,fp,205.00,88.00,152.00,66.00,154.00,203.00,260.00,233.00
4,two_stage_pred_label,fn,6.00,7.00,7.00,9.00,3.00,9.00,4.00,7.00
...,...,...,...,...,...,...,...,...,...,...
91,two_stage_95_label,fnr,4.44,27.59,13.21,64.29,15.79,35.71,8.00,15.79
92,two_stage_95_label,accuracy,47.00,76.00,60.00,81.00,60.00,44.00,29.00,37.00
93,two_stage_95_label,f1,61.98,30.88,36.65,11.90,17.02,14.88,25.92,35.26
94,two_stage_95_label,f05,51.19,22.98,27.22,8.50,11.51,10.18,18.11,26.14


In [294]:
df_metric.to_csv('df_metric.csv', index=False)

## Best weight According to df_metric

In [295]:
best_weight = pd.DataFrame(dict(prediction_type = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
                                two_stage_weight = [0.85,0.9,0.85,0.85,0.9,0.8,1],
                                ensemble_threshold = [0.43,0.1,0.48,0.37,0.35,0.51,0.42]))
best_weight

,prediction_type,two_stage_weight,ensemble_threshold
0,C1,0.85,0.43
1,C2,0.90,0.10
2,C3,0.85,0.48
3,C4,0.85,0.37
4,C5,0.90,0.35
5,C6,0.80,0.13
6,C7,1.00,0.42


## Final Ensemble Model

In [296]:
def ensemble(df):
    best_weight = pd.DataFrame(dict(prediction_type = [ 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
                                    two_stage_weight = [0.85,0.9,0.85,0.85,0.9,0.8,1],
                                   ensemble_threshold = [0.43,0.1,0.48,0.37,0.35,0.51,0.42]))


    df['two_stage_pred_label'] = np.where(df['two_stage_pred']>=df['two_stage_threshold'],1,0)
    df['effnet_pred_label'] = np.where(df['effnet_pred']>=df['effnet_threshold'],1,0)
   
    
    df = df.merge(best_weight, how='inner',
                                left_on = 'prediction_type', right_on = 'prediction_type')

    df['ensemble_pred'] = df['two_stage_pred']*df['two_stage_weight'] + df['effnet_pred']*(1-df['two_stage_weight'])

    df['ensemble_label'] = np.where(df['ensemble_pred']>=df['ensemble_threshold'],1,0)
    
    df_overall = df.groupby('StudyInstanceUID')[['label','two_stage_pred_label','effnet_pred_label','ensemble_label']].sum().reset_index()
    df_overall['label'] =  np.where(df_overall['label']>0,1,0)
    df_overall['two_stage_pred_label'] =  np.where(df_overall['two_stage_pred_label']>0,1,0)
    df_overall['effnet_pred_label'] =  np.where(df_overall['effnet_pred_label']>0,1,0)
    df_overall['ensemble_label'] =  np.where(df_overall['ensemble_label']>0,1,0)
    df_overall['prediction_type'] = 'patient_overall'

    df_full = pd.concat([df_overall, 
                         df[['StudyInstanceUID', 'label','two_stage_pred_label','effnet_pred_label','ensemble_label','prediction_type']]])

    return df_full

In [297]:
overall_infer = pd.read_csv('overall_infer.csv')
overall_infer_train_final = overall_infer[overall_infer['train'] == 1]
overall_infer_test_final = overall_infer[overall_infer['train'] == 0]

#### Result

In [298]:
ensemble_train = ensemble(overall_infer_train_final)
ensemble_train.to_csv('ensemble_train.csv', index=False)

In [299]:
ensemble_test = ensemble(overall_infer_test_final)
ensemble_test.to_csv('ensemble_test.csv', index=False)

In [300]:
ensemble_overall = ensemble(overall_infer)
ensemble_overall.to_csv('ensemble_overall.csv', index=False)

In [301]:
def ensemble_eval(df):
    model_type = []
    metric = []
    patient_overall = []
    c1 = []
    c2 = []
    c3 = []
    c4 = []
    c5 = []
    c6 = []
    c7 = []

    vertebrae = ['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
    label = ['two_stage_pred_label','effnet_pred_label','ensemble_label']
    metric_name = ['recall', 'precision', 'tn', 'fp', 'fn', 'tp', 'fpr', 'fnr', 'accuracy', 'f1', 'f05', 'f2']

    for j in label:

        metric_value_overall = define_eval_metrics(df[df['prediction_type']=='patient_overall'],'label', j)
        metric_value_c1 = define_eval_metrics(df[df['prediction_type']=='C1'],'label', j)
        metric_value_c2 = define_eval_metrics(df[df['prediction_type']=='C2'],'label', j)
        metric_value_c3 = define_eval_metrics(df[df['prediction_type']=='C3'],'label', j)
        metric_value_c4 = define_eval_metrics(df[df['prediction_type']=='C4'],'label', j)
        metric_value_c5 = define_eval_metrics(df[df['prediction_type']=='C5'],'label', j)
        metric_value_c6 = define_eval_metrics(df[df['prediction_type']=='C6'],'label', j)
        metric_value_c7 = define_eval_metrics(df[df['prediction_type']=='C7'],'label', j)

        for m in range(12):
            model_type.append(j)
            metric.append(metric_name[m])
            patient_overall.append(metric_value_overall[m])
            c1.append(metric_value_c1[m])
            c2.append(metric_value_c2[m])
            c3.append(metric_value_c3[m])
            c4.append(metric_value_c4[m])
            c5.append(metric_value_c5[m])
            c6.append(metric_value_c6[m])
            c7.append(metric_value_c7[m])


    df_metric = pd.DataFrame(dict(model_type=model_type,metric=metric,patient_overall=patient_overall,
                                 c1=c1,c2=c2,c3=c3,c4=c4,c5=c5,c6=c6,c7=c7))
    return df_metric

In [302]:
eval_train = ensemble_eval(ensemble_train)
eval_train.to_csv('ensemble_eval_train.csv', index=False)
eval_train

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,96.67,75.86,86.79,35.71,84.21,67.86,92.00,90.79
1,two_stage_pred_label,precision,45.91,20.00,23.23,7.04,9.41,8.56,15.03,22.85
2,two_stage_pred_label,tn,15.00,270.00,192.00,317.00,218.00,137.00,60.00,66.00
3,two_stage_pred_label,fp,205.00,88.00,152.00,66.00,154.00,203.00,260.00,233.00
4,two_stage_pred_label,fn,6.00,7.00,7.00,9.00,3.00,9.00,4.00,7.00
5,two_stage_pred_label,tp,174.00,22.00,46.00,5.00,16.00,19.00,46.00,69.00
6,two_stage_pred_label,fpr,93.18,24.58,44.19,17.23,41.40,59.71,81.25,77.93
7,two_stage_pred_label,fnr,3.33,24.14,13.21,64.29,15.79,32.14,8.00,9.21
8,two_stage_pred_label,accuracy,47.00,75.00,60.00,81.00,60.00,42.00,29.00,36.00
9,two_stage_pred_label,f1,62.25,31.65,36.65,11.76,16.93,15.20,25.84,36.51


In [303]:
eval_test = ensemble_eval(ensemble_test)
eval_test.to_csv('ensemble_eval_test.csv', index=False)
eval_test

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,95.56,66.67,90.00,50.00,50.00,100.00,100.00,88.46
1,two_stage_pred_label,precision,44.33,12.90,16.36,4.55,4.26,9.68,11.49,28.40
2,two_stage_pred_label,tn,1.00,65.00,43.00,77.00,49.00,30.00,8.00,11.00
3,two_stage_pred_label,fp,54.00,27.00,46.00,21.00,45.00,56.00,77.00,58.00
4,two_stage_pred_label,fn,2.00,2.00,1.00,1.00,2.00,0.00,0.00,3.00
5,two_stage_pred_label,tp,43.00,4.00,9.00,1.00,2.00,6.00,10.00,23.00
6,two_stage_pred_label,fpr,98.18,29.35,51.69,21.43,47.87,65.12,90.59,84.06
7,two_stage_pred_label,fnr,4.44,33.33,10.00,50.00,50.00,0.00,0.00,11.54
8,two_stage_pred_label,accuracy,44.00,70.00,53.00,78.00,52.00,39.00,19.00,36.00
9,two_stage_pred_label,f1,60.56,21.62,27.69,8.33,7.84,17.65,20.62,42.99


In [304]:
eval_overall = ensemble_eval(ensemble_overall)
eval_overall.to_csv('ensemble_eval_overall.csv', index=False)
eval_overall

,model_type,metric,patient_overall,c1,c2,c3,c4,c5,c6,c7
0,two_stage_pred_label,recall,96.44,74.29,87.30,37.50,78.26,73.53,93.33,90.20
1,two_stage_pred_label,precision,45.59,18.44,21.74,6.45,8.29,8.80,14.25,24.02
2,two_stage_pred_label,tn,16.00,335.00,235.00,394.00,267.00,167.00,68.00,77.00
3,two_stage_pred_label,fp,259.00,115.00,198.00,87.00,199.00,259.00,337.00,291.00
4,two_stage_pred_label,fn,8.00,9.00,8.00,10.00,5.00,9.00,4.00,10.00
5,two_stage_pred_label,tp,217.00,26.00,55.00,6.00,18.00,25.00,56.00,92.00
6,two_stage_pred_label,fpr,94.18,25.56,45.73,18.09,42.70,60.80,83.21,79.08
7,two_stage_pred_label,fnr,3.56,25.71,12.70,62.50,21.74,26.47,6.67,9.80
8,two_stage_pred_label,accuracy,47.00,74.00,58.00,80.00,58.00,42.00,27.00,36.00
9,two_stage_pred_label,f1,61.91,29.55,34.81,11.01,15.00,15.72,24.72,37.94
